In [0]:
#reading the csv file and converting it into spark dataframe adding schema
rawdf1=spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified",header=False,sep=',').toDF("id","firstname","lastname","age","profession")
rawdf1.show(20)
print("actual Record count",rawdf1.count())
rawdf1.printSchema()
rawdf1.describe()
print("Columns of rawdf1",rawdf1.columns)
print(rawdf1.dtypes)

rawdf1.summary()
#to view schema of rawdf1
print(rawdf1.schema)

#removing row level duplicates using distinct
print("distinct records in rawdf1",rawdf1.distinct().count())

#distinct & dropduplicates performs similar operations only, parameters cant be passed in distinct, but parameters can be passed in dropDuplicates
print("deduplication of records in rawdf1",rawdf1.dropDuplicates().count())

#removing column level duplicates using dropDuplicates by passing parameters
print("distinct records in rawdf1 after removing column level duplicates",rawdf1.dropDuplicates(['id']).count())
display(rawdf1.describe())
display(rawdf1.summary())
len(rawdf1.collect())







In [0]:
#importing spark session
from pyspark.sql.session import SparkSession
spark=SparkSession.builder.getOrCreate()

from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ShortType,

from pyspark.sql.functions import *

struct=StructType([StructField('id',StringType(), True),StructField('firstname',StringType(), True),StructField('lastname',StringType(), True),StructField('age',StringType(), True),StructField('profession',StringType(), True)])

#reading the csv file and converting it into spark dataframe adding schema
rawdf1=spark.read.schema(struct).csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified",sep=',',mode="permissive")
rawdf1.show(20)
rawdf1.count()

#----Active Data Munging----

#cleansing
#na.drop will remove the rows which has null values in all the columns, if subset is specified then records having nulls in the specified columns will be removed
cleanseddf1=rawdf1.na.drop(how='all',subset=["firstname","lastname"])
display(cleanseddf1.count())

#scrubbing

#na.fill will fill the nulls with provided values in specified coulmns
scrubbeddf1=cleanseddf1.na.fill("not provided",subset=["lastname","profession"])

#na.replace will replace a value with another in given subset
scrubbeddf2=scrubbeddf1.na.replace("not provided","NA",subset=["lastname"])
repl_dict_list={"Actor":"Celebrity","Pilot":"Captain"}
scrubbeddf3=scrubbeddf2.na.replace(repl_dict_list,subset=["profession"])
display(scrubbeddf3.show(10))

#De-duplication

#removing row level duplicate using distinct
dedupdf1=scrubbeddf3.distinct()
display(dedupdf1.show(10))


#removing column level duplicate using dropDuplicates
dedupdf2=dedupdf1.coalesce(1).dropDuplicates(['id'])
display(dedupdf2.show(10))
display(dedupdf2.where("id in ('4000001')"))

#Data Standardization

#Standardization 1 - Column Enrichment
stddf1=dedupdf2.withColumn("sourcesystem",lit("source"))
display(stddf1.show(10))

#standardization 2 - Column Transformation
stddf2 =stddf1.withColumn("profession",initcap(stddf1.profession))
display(stddf2.show(10))

display(stddf1.groupBy("profession").count())#DSL